# First BigQuery ML models for Taxifare Prediction

**Learning Objectives**
  * Choose the correct BigQuery ML model type and specify options
  * Evaluate the performance of your ML model
  * Improve model performance through data quality cleanup
  * Create a Deep Neural Network (DNN) using SQL

## Overview
In this notebook, we will use BigQuery ML to build our first models for taxifare prediction.BigQuery ML provides a fast way to build ML models on large structured and semi-structured datasets. We'll start by creating a dataset to hold all the models we create in BigQuery.

### Set environment variables

In [1]:
PROJECT = !gcloud config get-value project
PROJECT = PROJECT[0]
BUCKET = PROJECT
REGION = "us-central1"

%env PROJECT=$PROJECT
%env BUCKET=$BUCKET
%env REGION=$REGION

env: PROJECT=dsparing-sandbox
env: BUCKET=dsparing-sandbox
env: REGION=us-central1


## Create a BigQuery Dataset and Google Cloud Storage Bucket

A BigQuery dataset is a container for tables, views, and models built with BigQuery ML. Let's create one called __serverlessml__ if we have not already done so in an earlier lab. We'll do the same for a GCS bucket for our project too.

In [2]:
%%bash

# Create a BigQuery dataset for serverlessml if it doesn't exist
datasetexists=$(bq ls -d | grep -w serverlessml)

if [ -n "$datasetexists" ]; then
    echo -e "BigQuery dataset already exists, let's not recreate it."
else
    echo "Creating BigQuery dataset titled: serverlessml"

    bq --location=US mk --dataset \
        --description 'Taxi Fare' \
        $PROJECT:serverlessml
    echo "\nHere are your current datasets:"
    bq ls
fi    

# Create GCS bucket if it doesn't exist already...
exists=$(gsutil ls -d | grep -w gs://${BUCKET}/)

if [ -n "$exists" ]; then
    echo -e "Bucket exists, let's not recreate it."
else
    echo "Creating a new GCS bucket."
    gsutil mb -l ${REGION} gs://${BUCKET}
    echo "\nHere are your current buckets:"
    gsutil ls
fi

BigQuery dataset already exists, let's not recreate it.
Bucket exists, let's not recreate it.


## Model 1: Raw data

Let's build a model using just the raw data. It's not going to be very good, but sometimes it is good to actually experience this.

The model will take a minute or so to train. When it comes to ML, this is blazing fast.

In [3]:
%%bigquery
CREATE OR REPLACE MODEL
    serverlessml.model1_rawdata

OPTIONS(input_label_cols=['fare_amount'],
        model_type='linear_reg') AS

SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count * 1.0 AS passengers
FROM
    `nyc-tlc.yellow.trips`
WHERE
    ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 100000)) = 1

Query complete after 0.01s: 100%|██████████| 3/3 [00:00<00:00, 1060.06query/s]                        


""


Once the training is done, visit the [BigQuery Cloud Console](https://console.cloud.google.com/bigquery) and look at the model that has been trained. Then, come back to this notebook.

Note that BigQuery automatically split the data we gave it, and trained on only a part of the data and used the rest for evaluation. We can look at eval statistics on that held-out data:

In [4]:
%%bigquery
SELECT * FROM ML.EVALUATE(MODEL serverlessml.model1_rawdata)

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.41s/rows]


,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,6.086824,93.909516,0.32947,4.501958,0.00215,0.002582


Let's report just the error we care about, the Root Mean Squared Error (RMSE)

In [5]:
%%bigquery
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL serverlessml.model1_rawdata)

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.38s/rows]


,rmse
0,9.690692


We told you it was not going to be good!  Recall that our heuristic got 8.13, and our target is $6.

Note that the error is going to depend on the dataset that we evaluate it on.
We can also evaluate the model on our own held-out benchmark/test dataset, but we shouldn't make a habit of this (we want to keep our benchmark dataset as the final evaluation, not make decisions using it all along the way. If we do that, our test dataset won't be truly independent).

In [6]:
%%bigquery
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL serverlessml.model1_rawdata, (
    SELECT
        (tolls_amount + fare_amount) AS fare_amount,
        pickup_longitude AS pickuplon,
        pickup_latitude AS pickuplat,
        dropoff_longitude AS dropofflon,
        dropoff_latitude AS dropofflat,
        passenger_count * 1.0 AS passengers
    FROM
        `nyc-tlc.yellow.trips`
    WHERE
        ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 100000)) = 2
        AND trip_distance > 0
        AND fare_amount >= 2.5
        AND pickup_longitude > -78
        AND pickup_longitude < -70
        AND dropoff_longitude > -78
        AND dropoff_longitude < -70
        AND pickup_latitude > 37
        AND pickup_latitude < 45
        AND dropoff_latitude > 37
        AND dropoff_latitude < 45
        AND passenger_count > 0
    ))

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.17s/rows]


,rmse
0,9.428081


## Model 2: Apply data cleanup

Recall that we did some data cleanup in the previous lab. Let's do those before training.

This is a dataset that we will need quite frequently in this notebook, so let's extract it first.

In [7]:
%%bigquery
CREATE OR REPLACE TABLE
    serverlessml.cleaned_training_data AS

SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers
FROM
    `nyc-tlc.yellow.trips`
WHERE
    ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 100000)) = 1
    AND trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1514.55query/s]                        


""


In [8]:
%%bigquery
-- LIMIT 0 is a free query, this allows us to check that the table exists.
SELECT * FROM serverlessml.cleaned_training_data
LIMIT 0

Query complete after 0.00s: 100%|██████████| 1/1 [00:00<00:00, 490.73query/s]                          
Downloading: 0rows [00:00, ?rows/s]


,fare_amount,pickuplon,pickuplat,dropofflon,dropofflat,passengers


In [9]:
%%bigquery
CREATE OR REPLACE MODEL
    serverlessml.model2_cleanup

OPTIONS(input_label_cols=['fare_amount'],
        model_type='linear_reg') AS

SELECT
    *
FROM
    serverlessml.cleaned_training_data

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1230.60query/s]                        


""


In [10]:
%%bigquery
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL serverlessml.model2_cleanup)

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.50s/rows]


,rmse
0,7.919123


## Model 3: More sophisticated models

What if we try a more sophisticated model? Let's try Deep Neural Networks (DNNs) in BigQuery:

### DNN
To create a DNN, simply specify __dnn_regressor__ for the model_type and add your hidden layers.

In [11]:
%%bigquery
-- This training takes on the order of 15 minutes.
CREATE OR REPLACE MODEL
    serverlessml.model3b_dnn

OPTIONS(input_label_cols=['fare_amount'],
        model_type='dnn_regressor', hidden_units=[32, 8]) AS

SELECT
    *
FROM
    serverlessml.cleaned_training_data

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1615.26query/s]                        


""


In [12]:
%%bigquery
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL serverlessml.model3b_dnn)

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.39s/rows]


,rmse
0,4.680998


Nice!

## Evaluate DNN on benchmark dataset

Let's use the same validation dataset to evaluate -- remember that evaluation metrics depend on the dataset. You can not compare two models unless you have run them on the same withheld data.

In [13]:
%%bigquery
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL serverlessml.model3b_dnn, (
    SELECT
        (tolls_amount + fare_amount) AS fare_amount,
        pickup_datetime,
        pickup_longitude AS pickuplon,
        pickup_latitude AS pickuplat,
        dropoff_longitude AS dropofflon,
        dropoff_latitude AS dropofflat,
        passenger_count * 1.0 AS passengers,
        'unused' AS key
    FROM
        `nyc-tlc.yellow.trips`
    WHERE
        ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 10000)) = 2
        AND trip_distance > 0
        AND fare_amount >= 2.5
        AND pickup_longitude > -78
        AND pickup_longitude < -70
        AND dropoff_longitude > -78
        AND dropoff_longitude < -70
        AND pickup_latitude > 37
        AND pickup_latitude < 45
        AND dropoff_latitude > 37
        AND dropoff_latitude < 45
        AND passenger_count > 0
    ))

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.14s/rows]


,rmse
0,6.163327


Wow! Later in this sequence of notebooks, we will get to below $4, but this is quite good, for very little work.

In this notebook, we showed you how to use BigQuery ML to quickly build ML models. We will come back to BigQuery ML when we want to experiment with different types of feature engineering. The speed of BigQuery ML is very attractive for development.

Copyright 2019 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.